In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/Users/ohno/study/日経平均株価過去データ copy.csv")
df = df.drop(columns = ['出来高', '変化率 %'])
df = df.rename(columns={'日付け': 'Date', '終値': 'Close', '始値': 'Open', '高値': 'High', '安値': 'Low'})
df = df.reindex(columns=['Date', 'Open', 'High', 'Low', 'Close'])
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by = 'Date', ascending = True, inplace = True)
df = df.reset_index(drop=True)
df.dtypes

Date     datetime64[ns]
Open            float64
High            float64
Low             float64
Close           float64
dtype: object

In [6]:
def macd(df):
    FastEMA_period = 12  # 短期EMAの期間
    SlowEMA_period = 26  # 長期EMAの期間
    SignalSMA_period = 9  # SMAを取る期間
    df["MACD"] = df["Close"].ewm(span=FastEMA_period).mean() - df["Close"].ewm(span=SlowEMA_period).mean()
    df["Signal"] = df["MACD"].rolling(SignalSMA_period).mean()
    return df

def rsi(df):
    # 前日との差分を計算
    df_diff = df["Close"].diff(1)
 
    # 計算用のDataFrameを定義
    df_up, df_down = df_diff.copy(), df_diff.copy()
    
    # df_upはマイナス値を0に変換
    # df_downはプラス値を0に変換して正負反転
    df_up[df_up < 0] = 0
    df_down[df_down > 0] = 0
    df_down = df_down * -1
    
    # 期間14でそれぞれの平均を算出
    df_up_sma14 = df_up.rolling(window=14, center=False).mean()
    df_down_sma14 = df_down.rolling(window=14, center=False).mean()
 
    # RSIを算出
    df["RSI"] = 100.0 * (df_up_sma14 / (df_up_sma14 + df_down_sma14))
 
    return df

# MACDを計算する
df = macd(df)
 
# RSIを算出
df = rsi(df)

In [7]:
df = df.dropna()
df = df.reset_index(drop=True)
# df.to_csv('price.csv')
df

,Date,Open,High,Low,Close,MACD,Signal,RSI,Label
0,2001-01-26,13726.98,13750.23,13626.05,13696.06,39.969345,34.286122,45.323702,0
1,2001-01-29,13724.26,13908.40,13721.86,13845.28,39.570500,39.573369,56.799527,1
2,2001-01-30,13885.27,13910.71,13713.55,13826.65,37.622349,43.553451,62.570670,0
3,2001-01-31,13855.74,13855.74,13726.51,13843.55,36.734063,45.545389,73.752274,1
4,2001-02-01,13740.92,13779.55,13667.93,13779.55,31.499078,44.986421,67.003205,0
...,...,...,...,...,...,...,...,...,...
4980,2021-04-14,29718.06,29722.40,29567.18,29620.99,109.455309,106.458295,65.115498,0
4981,2021-04-15,29573.90,29787.66,29558.55,29642.69,103.346146,113.052752,59.236487,1
4982,2021-04-16,29789.08,29789.08,29621.83,29683.37,100.627160,113.333455,56.343906,1
4983,2021-04-19,29688.32,29808.01,29530.84,29685.37,97.509696,112.162026,55.470871,1
